##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Python 함수 구성 요소 튜토리얼


참고: 설정이 필요하지 않은 Colab 노트북에서 이 튜토리얼을 실행하는 것이 좋습니다! "Google Colab에서 실행"을 클릭하기만 하면 됩니다.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/python_function_component"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/python_function_component.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/python_function_component.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td>     <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/python_function_component.ipynb"><img width="32px" src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>
</td>
</table></div>

이 노트북에는 TFX InteractiveContext 및 로컬로 오케스트레이션된 TFX 파이프라인에서 Python 함수 구성 요소를 작성하고 실행하는 방법에 대한 예제가 포함되어 있습니다.

자세한 컨텍스트 및 정보는 TFX 문서 사이트의 [사용자 정의 Python 함수 구성 요소](https://www.tensorflow.org/tfx/guide/custom_function_component) 페이지를 참조하세요.

## !pip install -U tf-hub-nightly<br>import tensorflow_hub as hub<br><br>from tensorflow.keras import layers

먼저 TFX를 설치하고 필요한 모듈을 가져옵니다. TFX에는 Python 3이 필요합니다.

### Python 버전 확인하기


In [ ]:
import sys
sys.version

### Pip 업그레이드

로컬에서 실행할 때 시스템에서 Pip을 업그레이드하지 않으려면 Colab에서 실행 중인지 확인하세요. 물론 로컬 시스템은 별도로 업그레이드할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX 설치

**참고: Google Colab에서는 패키지 업데이트로 인해 이 셀을 처음 실행할 때 런타임을 다시 시작해야 합니다(런타임 &gt; 런타임 다시 시작...).**

In [ ]:
!pip install tfx

## 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, 위의 셀을 처음 실행할 때 런타임을 다시 시작해야 합니다(런타임 &gt; 런타임 다시 시작...). 이는 Colab이 패키지를 로드하는 방식 때문입니다.

### 패키지 가져오기

TFX를 가져오고 TFX의 버전을 확인합니다.

In [ ]:
# Check version
from tfx import v1 as tfx
tfx.__version__

## 사용자 정의 Python 함수 구성 요소

이 섹션에서는 Python 함수로부터 구성 요소를 생성합니다. 실제 ML 문제는 수행하지 않을 것입니다. 이 간단한 함수는 Python 함수 구성 요소 개발 프로세스의 설명에 사용됩니다.

자세한 문서는 [Python 함수 기반 구성 요소 가이드](https://www.tensorflow.org/tfx/guide/custom_function_component)를 참조하세요.

### Python 사용자 정의 구성 요소 생성하기

더미 데이터를 생성하는 함수를 작성하는 것으로 시작해보겠습니다. 이 함수는 자체 Python 모듈 파일에 작성됩니다.

In [ ]:
%%writefile my_generator.py

import os
import tensorflow as tf  # Used for writing files.

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset

@tfx.dsl.components.component
def MyGenerator(data: tfx.dsl.components.OutputArtifact[Dataset]):
  """Create a file with dummy data in the output artifact."""
  with tf.io.gfile.GFile(os.path.join(data.uri, 'data_file.txt'), 'w') as f:
    f.write('Dummy data')

  # Set metadata and ensure that it gets passed to downstream components.
  data.set_string_custom_property('my_custom_field', 'my_custom_value')

다음으로 생성된 더미 데이터를 사용하는 두 번째 구성 요소를 작성합니다. 데이터의 해시를 계산하고 이를 반환할 것입니다.

In [ ]:
%%writefile my_consumer.py

import hashlib
import os
import tensorflow as tf

from tfx import v1 as tfx

# Non-public APIs, just for showcase.
from tfx.types.experimental.simple_artifacts import Dataset
from tfx.types.standard_artifacts import String

@tfx.dsl.components.component
def MyConsumer(data: tfx.dsl.components.InputArtifact[Dataset],
               hash: tfx.dsl.components.OutputArtifact[String],
               algorithm: tfx.dsl.components.Parameter[str] = 'sha256'):
  """Reads the contents of data and calculate."""
  with tf.io.gfile.GFile(
      os.path.join(data.uri, 'data_file.txt'), 'r') as f:
    contents = f.read()
  h = hashlib.new(algorithm)
  h.update(tf.compat.as_bytes(contents))
  hash.value = h.hexdigest()

  # Read a custom property from the input artifact and set to the output.
  custom_value = data.get_string_custom_property('my_custom_field')
  hash.set_string_custom_property('input_custom_field', custom_value)

### InteractiveContext를 사용하여 노트북 내에서 실행하기

이제 TFX InteractiveContext에서 새 구성 요소를 사용하는 방법을 설명합니다.

TFX 노트북 InteractiveContext로 수행할 수 있는 작업에 대한 자세한 정보는 노트북 내 [TFX Keras 구성 요소 가이드](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras)를 참조하세요.

In [ ]:
from my_generator import MyGenerator
from my_consumer import MyConsumer

#### InteractiveContext 구성하기

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

#### `context.run()`으로 구성 요소를 대화형으로 실행하기

다음으로 `context.run()`을 사용하여 노트북 내에서 구성 요소를 대화식으로 실행합니다. 소비자 구성 요소는 생성기 구성 요소의 출력을 사용합니다.

In [ ]:
generator = MyGenerator()
context.run(generator)

In [ ]:
consumer = MyConsumer(
    data=generator.outputs['data'],
    algorithm='md5')
context.run(consumer)

실행한 후에 디스크에 있는 소비자 구성 요소의 "해시" 출력 아티팩트의 내용을 검사할 수 있습니다.

In [ ]:
!tail -v {consumer.outputs['hash'].get()[0].uri}

이게 전부입니다! 지금까지 사용자 정의 구성 요소를 작성하고 실행하는 작업을 진행했습니다.

### 파이프라인 정의 작성하기

다음으로, 동일한 구성 요소를 사용하여 파이프라인을 작성합니다. 노트북 내에서 `InteractiveContext`를 사용하면 실험에 적합하게 되지만 파이프라인을 정의하면 프로덕션 사용을 위해 로컬 또는 원격 실행기에 파이프라인을 배포할 수 있습니다.

여기에서는 여러분의 장치에서 로컬로 실행되는 LocalDagRunner의 사용법을 설명합니다. 프로덕션 실행의 경우 Airflow 또는 Kubeflow 실행기가 더 적합할 수 있습니다.

#### 파이프라인 구성하기

In [ ]:
import os
import tempfile
from tfx import v1 as tfx

# Select a persistent TFX root directory to store your output artifacts.
# For demonstration purposes only, we use a temporary directory.
PIPELINE_ROOT = tempfile.mkdtemp()
# Select a pipeline name so that multiple runs of the same logical pipeline
# can be grouped.
PIPELINE_NAME = "function-based-pipeline"
# We use a ML Metadata configuration that uses a local SQLite database in
# the pipeline root directory. Other backends for ML Metadata are available
# for production usage.
METADATA_CONNECTION_CONFIG = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    os.path.join(PIPELINE_ROOT, 'metadata.sqlite'))

def function_based_pipeline():
  # Here, we construct our generator and consumer components in the same way.
  generator = MyGenerator()
  consumer = MyConsumer(
      data=generator.outputs['data'],
      algorithm='md5')

  return tfx.dsl.Pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      components=[generator, consumer],
      metadata_connection_config=METADATA_CONNECTION_CONFIG)

my_pipeline = function_based_pipeline()

#### `LocalDagRunner`로 파이프라인 실행하기

In [ ]:
tfx.orchestration.LocalDagRunner().run(my_pipeline)

이 파이프라인 실행으로 생성된 출력 아티팩트를 검사할 수 있습니다.

In [ ]:
!find {PIPELINE_ROOT}

지금까지 사용자 정의 구성 요소를 작성하고 LocalDagRunner에서의 실행을 오케스트레이션했습니다! 다음 단계로 [TFX 웹사이트](https://www.tensorflow.org/tfx)에서 추가 튜토리얼과 가이드를 확인하세요.